In [10]:
import pandas as pd
import numpy as np
dataset=pd.read_csv("ckd.csv")


In [11]:
dataset=pd.get_dummies(dataset, drop_first=True)
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [12]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]



In [13]:

dependent=dataset['classification_yes']


In [18]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [19]:
#from sklearn.preprocessing import StandardScaler
#SC=StandardScaler()
#x_train=SC.fit_transform(x_train)
#x_test=SC.transform(x_test)
#x_train

In [20]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid= {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]}

grid=GridSearchCV(ComplementNB(),param_grid,scoring='f1_weighted')
grid.fit(x_train,y_train)

GridSearchCV(estimator=ComplementNB(),
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]},
             scoring='f1_weighted')

In [21]:
# print best parameter after tuning 
#print(grid.best_params_) 


In [22]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00577197, 0.0054678 , 0.00488272, 0.00440044, 0.00455322,
       0.00442891, 0.00496349]), 'std_fit_time': array([0.0031058 , 0.00093453, 0.0002391 , 0.00080081, 0.00042234,
       0.00053153, 0.00017216]), 'mean_score_time': array([0.00498672, 0.0040215 , 0.00384469, 0.00380497, 0.00399337,
       0.00398626, 0.00408058]), 'std_score_time': array([6.08295229e-04, 5.97920839e-04, 3.09801340e-04, 7.02443292e-04,
       1.64813299e-05, 7.86118662e-05, 1.37126992e-04]), 'param_alpha': masked_array(data=[0.1, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'alpha': 0.1}, {'alpha': 0.5}, {'alpha': 1.0}, {'alpha': 1.5}, {'alpha': 2.0}, {'alpha': 5.0}, {'alpha': 10.0}], 'split0_test_score': array([0.83592759, 0.81736909, 0.81736909, 0.79860872, 0.79860872,
       0.79860872, 0.77960928]), 'split1_test_score': array([0.8515274 , 0.8515274 , 0.8515274 , 0

In [23]:
grid_predictions = grid.predict(x_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
cm


array([[50,  1],
       [20, 62]], dtype=int64)

In [24]:

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.71      0.98      0.83        51
           1       0.98      0.76      0.86        82

    accuracy                           0.84       133
   macro avg       0.85      0.87      0.84       133
weighted avg       0.88      0.84      0.84       133



In [25]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 0.1}: 0.8441571297865639


In [26]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])


0.9356767097082734

In [27]:
table=pd.DataFrame.from_dict(re)

In [28]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005772,0.003106,0.004987,0.000608,0.1,{'alpha': 0.1},0.835928,0.851527,0.888515,0.888515,0.888286,0.870554,0.022453,1
1,0.005468,0.000935,0.004022,0.000598,0.5,{'alpha': 0.5},0.817369,0.851527,0.869925,0.888515,0.869849,0.859437,0.024067,2
2,0.004883,0.000239,0.003845,0.000310,1,{'alpha': 1.0},0.817369,0.851527,0.869925,0.869925,0.851354,0.852020,0.019197,3
3,0.004400,0.000801,0.003805,0.000702,1.5,{'alpha': 1.5},0.798609,0.833323,0.869925,0.869925,0.851354,0.844627,0.026723,4
4,0.004553,0.000422,0.003993,0.000016,2,{'alpha': 2.0},0.798609,0.833323,0.851225,0.851225,0.851354,0.837147,0.020484,5
5,0.004429,0.000532,0.003986,0.000079,5,{'alpha': 5.0},0.798609,0.796687,0.851225,0.851225,0.851354,0.829820,0.026276,6
6,0.004963,0.000172,0.004081,0.000137,10,{'alpha': 10.0},0.779609,0.778179,0.851225,0.794081,0.851354,0.810890,0.033452,7
